# OECD Data Intensity Pipeline
This notebook implements the full pipeline for analyzing the "Data Intensity" of job advertisements.
It follows the methodology described in the OECD 2023 paper *The output and investment of the data-driven economy*.

**Core Steps:**
1.  **Ingestion**: Loading around 60 million job ads when full data (2020-2025).
2.  **NLP Analysis**: Using SpaCy to extract noun chunks and compute similarity to the concept of "data".
3.  **Classification**: Tagging jobs as 'Data Entry', 'Database', or 'Data Analytics'.
4.  **Economic Valuation**: Mapping job ads to industries (SIC) and applying the OECD "Alpha" parameters to estimate Data Investment as a % of GVA.



## Section 1: Environment Setup & Basic Configuration
All imports and core dependencies for the pipeline.

In [46]:

import pyspark.sql.functions as F
from pyspark.sql.types import *
import os
import datetime
from tqdm.auto import tqdm

from pyspark.sql import SparkSession

# Data manipulation and visualization
import pandas as pd
import plotly.express as px



### 1.1: Spark Session Initialization
Initializes or reuses an active Spark session with optimized configuration:
- **Reuse**: If Spark session already exists (from previous run), it's reused
- **Create**: Otherwise, a new session is created with optimized settings for large-scale NLP processing
- **Config**: Includes 13GB executor memory, Adaptive Query Execution (AQE), Kryo serialization, and Arrow for efficient Pandas conversion

In [47]:
# SPARK SESSION — Create or reuse
# This block handles both cases: reuse of existing session or creation with optimized config

try:
    spark  # Try to access existing session (noqa: F821)
    spark.sparkContext.setLogLevel("ERROR")
    print("✓ Re-using existing Spark session.")
except NameError:
    # Create optimized session for large-scale NLP
    spark = (
        SparkSession.builder
        .appName("OECD_Data_Intensity_Pipeline")
        .config("spark.executor.memory", "13g")
        .config("spark.driver.memory", "3g")
        .config("spark.executor.cores", "4")
        .config("spark.driver.cores", "4")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
        .config("spark.sql.adaptive.skewJoin.enabled", "true")
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("ERROR")
    print("✓ Spark session created with optimized config.")


✓ Re-using existing Spark session.


### 1.2: Pipeline Parameters Configuration
Define global controls for the entire pipeline. These settings determine:
- **Data Range**: `YEARS` specifies which years to process (2020-2025)
- **Sampling**: `SAMPLE_FRACTION` controls testing vs. production (1.0 = full data)
- **NLP Thresholds**: `SIM_THRESHOLD` and `DATA_THRESHOLD` control data-intensity classification
- **Recomputation**: `FORCE_RECOMPUTE` determines whether to skip existing year outputs

In [48]:
# ---------------------------
# Pipeline parameters
# ---------------------------
# "ALL" will auto-detect all years, or specify range for production
YEARS = "2020-2025"   

# Sampling: Set to 1.0 (100%) for your full production run
SAMPLE_FRACTION = 1.0      
SAMPLE_SEED = 42
STRATIFIED_BY_SOC = False   

# NLP thresholds (OECD-style)
# These remain standard
#it has to be 3 and 0.45 respectively
SIM_THRESHOLD = 0.45       
DATA_THRESHOLD = 3        

# How much to store per job
# Keep this at 10. Increasing it (e.g., to 100) will bloat storage on 240M rows.

# Re-run behaviour
# Set to False if you want to be able to restart a failed run without re-doing finished years
FORCE_RECOMPUTE = True      

# Debugging
WRITE_DEBUG_CHUNKS = False   
DEBUG_SAMPLE_FRACTION = 0.0001

print("Production parameters set.")

Production parameters set.


### 1.3: File Paths & Directory Structure
Centralizes all file and directory paths for the pipeline. This cell establishes:
- **Input Paths**: CSV data locations for raw job advertisements
- **Output Paths**: Versioned parquet output directories for each processing step
- **Helper Functions**: Path validation, year parsing, and path generation for all years
- **Path Organization**: Ensures all data is stored within `BASE_S3A_PATH` with consistent year-based subdirectories

In [49]:
# Root directory: keep everything inside OECD_DATA
BASE_S3A_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD"  # e.g. "s3a://onscdp-prd-data01-d4946922/dapsen/workspace_zone/online_job_ads/OECD_DATA"

# If you ever want to read from a single parquet instead of CSV partitions
PARQUET_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/part-00000-6f2787d8-9f9c-4b9b-9903-fc9d83e3d0c0-c000.snappy.parquet"  # e.g. f"{BASE_S3A_PATH}/somefile.parquet"
CSV_PATH = None      # if None, uses BASE_S3A_PATH/csv_data/<year>

In [50]:
def get_paths_for_year(year: int):
    base_year_path = os.path.join(BASE_S3A_PATH, "processed_data", str(year))

    if CSV_PATH:
        input_csv = os.path.join(CSV_PATH, str(year))
    else:
        input_csv = os.path.join(BASE_S3A_PATH, "csv_data", str(year))

    return {
        "input_csv": input_csv,


        # NLP processing outputs
        "noun_chunks": os.path.join(base_year_path, "noun_chunks"),
        "reduced_data": os.path.join(base_year_path, "reduced_data"),
        
        # existing outputs
        "job_features": os.path.join(base_year_path, "job_features"),
        "debug_chunks": os.path.join(base_year_path, "debug_chunks"),
        "job_categories": os.path.join(base_year_path, "job_categories"),
        "occupation_summary": os.path.join(base_year_path, "occupation_summary"),
        "job_categories_sic": os.path.join(base_year_path, "job_categories_sic"),      # synthetic
        "sector_summary_sic": os.path.join(base_year_path, "sector_summary_sic"),      # synthetic

        # NEW: census-based SIC allocation outputs
        "job_categories_sic_census": os.path.join(base_year_path, "job_categories_sic_census"),
        "sector_summary_sic_census": os.path.join(base_year_path, "sector_summary_sic_census"),
    }


In [51]:


def get_available_years():
    return list(range(2020, 2026))

def parse_year_input(year_input):
    available = get_available_years()
    if isinstance(year_input, str):
        y = year_input.strip().upper()
        if y == "ALL":
            years = available
        elif "-" in y:
            s, e = map(int, y.split("-"))
            years = list(range(s, e + 1))
        else:
            years = [int(y)]
    elif isinstance(year_input, (list, tuple)):
        years = [int(x) for x in year_input]
    else:
        years = [int(year_input)]
    bad = [yy for yy in years if yy not in available]
    if bad:
        raise ValueError(f"Invalid years {bad}. Available: {available}")
    return sorted(years)

years_to_process = parse_year_input(YEARS)
print("Years to process:", years_to_process)

def validate_oecd_path(path: str) -> bool:
    if not path.startswith(BASE_S3A_PATH):
        raise ValueError(f"Path {path} is outside OECD_DATA root")
    return True


ALL_PATHS = {yr: get_paths_for_year(yr) for yr in years_to_process}

# Validate
for yr, paths in ALL_PATHS.items():
    for k, p in paths.items():
        validate_oecd_path(p)

print("Path validation complete.")
print("Example year paths:", years_to_process[0], ALL_PATHS[years_to_process[0]])

Years to process: [2020, 2021, 2022, 2023, 2024, 2025]
Path validation complete.
Example year paths: 2020 {'input_csv': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/csv_data/2020', 'noun_chunks': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/noun_chunks', 'reduced_data': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/reduced_data', 'job_features': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_features', 'debug_chunks': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/debug_chunks', 'job_categories': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories', 'occupation_summary': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/occupation_summary', 'job_categories_sic': '/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet

### 1.4: Utility Functions
Helper functions for safe I/O operations and data versioning:
- **`safe_write_parquet_s3a()`**: Writes data to timestamped versioned folders (`_v=YYYYMMDD_HHMMSS`) with automatic pointer updates for atomic version tracking
- **`read_latest_version()`**: Reads the most recent successful output by following the `_LATEST_POINTER` marker
- These enable atomic updates and full history tracking for reproducibility

In [52]:
def safe_write_parquet_s3a(df, output_base_path, mode="overwrite", create_version=True, verify_read=True):
    """
    Safe-ish write for object stores (s3a://).
    Writes to version folder output_base_path/_v=YYYYMMDD_HHMMSS and updates a _LATEST_POINTER file.
    """
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    version_path = f"{output_base_path}/_v={ts}" if create_version else output_base_path

    print(f"[WRITE] parquet -> {version_path}")
    df.write.mode(mode).option("compression", "snappy").parquet(version_path)

    if verify_read:
        _ = spark.read.parquet(version_path).limit(1).count()

    # marker (optional)
    try:
        marker_df = spark.createDataFrame([(ts,)], ["written_at"])
        marker_df.coalesce(1).write.mode("overwrite").json(f"{version_path}/_SUCCESS_MARKER")
    except Exception as e:
        print(f"Warning: marker write failed: {e}")

    # pointer
    if create_version:
        try:
            latest_df = spark.createDataFrame([(version_path, ts)], ["latest_path", "timestamp"])
            latest_df.coalesce(1).write.mode("overwrite").json(f"{output_base_path}/_LATEST_POINTER")
        except Exception as e:
            print(f"Warning: pointer update failed: {e}")

    return version_path

def read_latest_version(output_base_path):
    pointer_path = f"{output_base_path}/_LATEST_POINTER"
    pointer = spark.read.json(pointer_path)
    latest = pointer.orderBy(F.col("timestamp").desc()).limit(1).collect()[0]["latest_path"]
    return spark.read.parquet(latest), latest

print("Utilities ready: safe_write_parquet_s3a(), read_latest_version()")

Utilities ready: safe_write_parquet_s3a(), read_latest_version()


## Section 2: Data Loading & Preparation
### 2.1: Load Raw Job Advertisement Data
Loads the raw job advertisement data from CSV or Parquet sources with optional sampling:
- **Supports CSV and Parquet**: Flexible input format handling
- **Year-based Partitioning**: Loads data by year for parallel processing
- **Sampling**: Optionally samples data for testing (set `SAMPLE_FRACTION < 1.0`)
- **Stratification**: Can preserve rare occupation categories during sampling if needed

In [53]:
all_data = {}
total_jobs = 0

if PARQUET_PATH:
    print(f"[LOAD] unified parquet from {PARQUET_PATH}")
    full_df = spark.read.parquet(PARQUET_PATH).withColumn("date", F.to_date("date"))

    for yr in tqdm(years_to_process, desc="Reading data (Parquet)"):
        df = full_df.filter(F.year("date") == yr)

        if SAMPLE_FRACTION is not None and float(SAMPLE_FRACTION) < 1.0:
            frac = float(SAMPLE_FRACTION)
            df = df.sample(False, frac, seed=SAMPLE_SEED)
            print(f"[LOAD] Year {yr}: sampled {frac*100:.2f}%")

        cnt = df.count()
        all_data[yr] = df
        total_jobs += cnt
        print(f"[LOAD] Year {yr}: {cnt:,} rows")

else:
    for yr in tqdm(years_to_process, desc="Reading data (CSV)"):
        print(f"\n[LOAD] Year {yr} from {ALL_PATHS[yr]['input_csv']} ...")

        df = (
            spark.read
            .option("header", True)
            .option("multiline", True)
            .csv(ALL_PATHS[yr]["input_csv"])
            .select(
                "date",
                "job_id",
                "soc_2020",
                "job_title",
                F.col("full_text").cast("string")
            )
            .filter(F.col("full_text").isNotNull() & (F.length("full_text") > 0))
            .withColumn("date", F.to_date("date", "yyyy-MM-dd"))
        )

        # Sampling
        if SAMPLE_FRACTION is not None and float(SAMPLE_FRACTION) < 1.0:
            frac = float(SAMPLE_FRACTION)

            if STRATIFIED_BY_SOC:
                # approximate stratified sample
                soc_vals = [r["soc_2020"] for r in df.select("soc_2020").distinct().limit(50000).collect()]
                fractions = {s: frac for s in soc_vals if s is not None}
                df = df.sampleBy("soc_2020", fractions=fractions, seed=SAMPLE_SEED)
                print(f"[LOAD] Year {yr}: stratified sample {frac*100:.2f}% by SOC")
            else:
                df = df.sample(False, frac, seed=SAMPLE_SEED)
                print(f"[LOAD] Year {yr}: uniform sample {frac*100:.2f}%")

        cnt = df.count()
        all_data[yr] = df
        total_jobs += cnt
        print(f"[LOAD] Year {yr}: {cnt:,} rows")

print(f"\n[LOAD] Total jobs loaded: {total_jobs:,}")

[LOAD] unified parquet from /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/part-00000-6f2787d8-9f9c-4b9b-9903-fc9d83e3d0c0-c000.snappy.parquet


Reading data (Parquet):   0%|          | 0/6 [00:00<?, ?it/s]

[LOAD] Year 2020: 1,755 rows
[LOAD] Year 2021: 1,633 rows


Reading data (Parquet):  50%|█████     | 3/6 [00:00<00:00, 25.44it/s]

[LOAD] Year 2022: 1,691 rows
[LOAD] Year 2023: 1,710 rows


Reading data (Parquet): 100%|██████████| 6/6 [00:00<00:00, 26.60it/s]

[LOAD] Year 2024: 1,600 rows
[LOAD] Year 2025: 1,611 rows

[LOAD] Total jobs loaded: 10,000


## Section 3: NLP Processing & Data Classification
### 3.1: NLP Feature Extraction & Data-Intensity Classification
Applies SpaCy NLP to extract noun chunks and classify jobs by data intensity:
- **Noun Chunk Extraction**: Extracts all noun phrases from job descriptions
- **Semantic Similarity**: Computes cosine similarity to "data" concept vector
- **Classification**: Tags jobs as "Data Entry", "Database", or "Data Analytics"
- **Caching**: Results saved to versioned `job_features` and `job_categories` directories
- **Efficiency**: Uses Spark's Arrow for fast Pandas UDF execution

In [54]:
import spacy
import pandas as pd
import gc
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType


# =========================================================
# 2. SCHEMA DEFINITION
# =========================================================
noun_schema = StructType([
    StructField("doc_date", StringType()),
    StructField("doc_JobID", StringType()),
    StructField("doc_BGTOcc", StringType()),
    StructField("noun_chunk", StringType()),
    StructField("sim_data", DoubleType())
])

# =========================================================
# 3. WORKER FUNCTION
# =========================================================

def extract_noun_chunks(iterator):
    try:
        nlp = spacy.load("en_core_web_lg", disable=["lemmatizer", "ner"])
    except OSError:
        nlp = spacy.load("en_core_web_sm")

    target = nlp("data")
    for pdf in iterator:
        rows = []
        texts = pdf["full_text"].fillna("").astype(str).tolist()
        jobids = pdf["job_id"].astype(str).tolist()
        dates = pdf["date"].astype(str).tolist()
        socs = pdf["soc_2020"].astype(str).tolist()

        #smaller batch size = less memory per task
        for i, doc in enumerate(nlp.pipe(texts,batch_size=25,n_process=1)):
            for chunk in doc.noun_chunks:
                if chunk.has_vector:
                   sim = float(chunk.similarity(target))
                   cleaned = "".join(c for c in chunk.text if not c.isdigit()).strip()
                   if cleaned:
                       rows.append({
                           'doc_date': dates[i],
                           'doc_JobID': jobids[i],
                           'doc_BGTOcc': socs[i],
                           'noun_chunk': cleaned.lower(),
                           'sim_data': sim,
                       })
        
        if rows:
            yield pd.DataFrame(rows)
        else:
            yield pd.DataFrame(columns=["doc_date", "doc_JobID", "doc_BGTOcc", "noun_chunk", "sim_data"])

SIM_THRESHOLD = 0.45


for yr in years_to_process:
    print(f"\n[PROCESS] Extracting noun chunks for year {yr} ...")

    out_path = ALL_PATHS[yr]["noun_chunks"]
    if not FORCE_RECOMPUTE:
        try:
            spark.read.parquet(out_path)
            print(f'\t[skip] Noun chunks already exist for {yr}')
            continue
        except:
            pass



    df = all_data[yr]

    estimated_row_size = 1000
    total_rows = df.count()
    target_partition_size = 128 * 1024 * 1024
    num_partitions = max(8, int((total_rows * estimated_row_size) / target_partition_size))

    if total_jobs > 15000000:
        num_partitions = int(num_partitions * 1.5)
    df_part = df.repartition(num_partitions)

    nc_df = df_part.mapInPandas(extract_noun_chunks, schema=noun_schema)
    nc_df.cache()

    nc_df.write.mode("overwrite").partitionBy("doc_date").parquet(out_path)
    total_chunks = nc_df.count()
    
    uniq_jobs = nc_df.select("doc_JobID").distinct().count()
    print(f'\t Saved noun chunks to {out_path} (chunks: {total_chunks:,}, jobs: {uniq_jobs:,})')
    nc_df.unpersist()


    del df_part, nc_df
    gc.collect()
    spark.catalog.clearCache()




[PROCESS] Extracting noun chunks for year 2020 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/noun_chunks (chunks: 6,153, jobs: 1)

[PROCESS] Extracting noun chunks for year 2021 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2021/noun_chunks (chunks: 5,719, jobs: 1)

[PROCESS] Extracting noun chunks for year 2022 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2022/noun_chunks (chunks: 5,915, jobs: 1)

[PROCESS] Extracting noun chunks for year 2023 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/noun_chunks (chunks: 5,985, jobs: 1)

[PROCESS] Extracting noun chunks for year 2024 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/noun_chunks (chunks: 5,593, jobs: 1)

[PROCESS] Extracting noun chunks for year 2025 ...


	 Saved noun chunks to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/noun_chunks (chunks: 5,649, jobs: 1)


In [55]:
for yr in tqdm(years_to_process, desc="Reducing noun chunks to job features"):
    print(f'\n Reducing data for year {yr} ...')

    out_path = ALL_PATHS[yr]["job_features"]
    if not FORCE_RECOMPUTE:
        try:
            spark.read.parquet(out_path)
            print(f'\t[skip] Job features already exist for {yr}')
            continue
        except:
            pass
    
    nc_path =ALL_PATHS[yr]["noun_chunks"]
    noun_chunks_df = spark.read.parquet(nc_path)

    #fileter by similarity threshold first (reduce data immediately)
    filtered = noun_chunks_df.filter(F.col("sim_data") >= SIM_THRESHOLD)

    #calculate aggregate stats Without collect_list (memory efficient)

    job_stats = (
        filtered.groupBy("doc_date", "doc_JobID", "doc_BGTOcc")
        .agg(
            F.count("*").alias('n_chunks_data'),
            F.avg("sim_data").alias("avg_sim_data"),
        )
    )

    #collect all chunks per job (orderd by similarity descending)
    top_chunks_df = (
        filtered.groupBy("doc_date", "doc_JobID", "doc_BGTOcc")
        .agg(
            F.collect_list("noun_chunk").alias("top_chunks"),
            F.collect_list("sim_data").alias("top_sims"),
             )
    )     

    # Join stats with top chunks
job_features_final = (
    job_stats.join(
        top_chunks_df,
        on=['doc_date', 'doc_JobID', 'doc_BGTOcc'],
        how='left'
    )
)

# Rename columns first
job_features_final = (
    job_features_final.withColumnRenamed('doc_date', 'date')
    .withColumnRenamed('doc_JobID', 'job_id')
    .withColumnRenamed('doc_BGTOcc', 'soc_2020')
    .withColumn('n_chunks_total', F.col('n_chunks_data'))
)

# Add year/month columns (now that date column is renamed)
job_features_final = (
    job_features_final.withColumn('year', F.year(F.to_date('date')))
    .withColumn('month', F.month(F.to_date('date')))
)

# Handle null top_chunks/top_sims by replacing with empty arrays
job_features_final = (
    job_features_final.withColumn('top_chunks', F.when(F.col('top_chunks').isNull(), F.array()).otherwise(F.col('top_chunks')))
    .withColumn('top_sims', F.when(F.col('top_sims').isNull(), F.array()).otherwise(F.col('top_sims')))
)

# Select final output columns in correct order
job_features_final = job_features_final.select(
    'date', 'job_id', 'soc_2020', 'n_chunks_total', 'n_chunks_data',
    'avg_sim_data', 'top_chunks', 'top_sims', 'year', 'month'
)

# Write job features
job_features_final.write.mode('overwrite').parquet(out_path)

total_jobs = job_features_final.count()
avg_chunks = job_features_final.agg(F.avg('n_chunks_data')).collect()[0][0]
print(f'\tSaved job features to {out_path} (jobs: {total_jobs:,}, avg chunks per job: {avg_chunks:.1f})')

# Aggressive cleanup for memory management
del filtered, job_stats, top_chunks_df, job_features_final
gc.collect()
spark.catalog.clearCache()
    


Reducing noun chunks to job features:   0%|          | 0/6 [00:00<?, ?it/s]


 Reducing data for year 2020 ...


Reducing noun chunks to job features:  17%|█▋        | 1/6 [00:00<00:02,  1.90it/s]


 Reducing data for year 2021 ...


Reducing noun chunks to job features:  33%|███▎      | 2/6 [00:01<00:02,  1.77it/s]

...


 Reducing data for year 2024 ...


Reducing noun chunks to job features:  83%|████████▎ | 5/6 [00:02<00:00,  1.97it/s]


 Reducing data for year 2025 ...


Reducing noun chunks to job features: 100%|██████████| 6/6 [00:03<00:00,  1.90it/s]


	Saved job features to /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/job_features (jobs: 807, avg chunks per job: 3.0)


### 3.2: SOC-Level (Occupation) Aggregation
Aggregates job classification data to the 4-digit SOC (Standard Occupational Classification):
- **SOC4 Grouping**: Summarizes data-intensive job percentages by occupation code
- **Data Counts**: Tracks total, data-entry, database, and data-analytics job counts per occupation
- **Shares**: Computes percentages for each data category by occupation
- **Output Structure**: Saves to `occupation_summary/` with versioning

In [56]:
# Landmark SOC4 groups
data_entry_soc = {"4111","4112","4113","4114","4121","4131","4132","4150"}
database_soc = {"2423","2136"}
data_analytics_soc = {"2421","2424","2133","2135"}

for yr in tqdm(years_to_process, desc="Job classification + occ summary"):
    print(f"\n[CLASSIFY] Year {yr}")

    # Read the features we just created
    job_feat_path = ALL_PATHS[yr]["job_features"]
    job_feat_df = spark.read.parquet(job_feat_path)

    
    # Ensure we only process this year's data
    job_feat_df = job_feat_df.filter(F.year(F.to_date("date")) == yr)
    
    # Extract SOC4 (First 4 characters)
    job_feat_df = job_feat_df.withColumn("soc4", F.substring(F.col("soc_2020"), 1, 4))

    # Apply Threshold Rule
    is_data_intensive = (F.col("n_chunks_data") >= F.lit(DATA_THRESHOLD))

    # Create Categories
    job_categories = (
        job_feat_df.select("job_id","soc_2020","soc4","date","n_chunks_data","avg_sim_data","top_chunks","top_sims")
        .withColumn("data_entry", (F.col("soc4").isin(list(data_entry_soc)) & is_data_intensive).cast("int"))
        .withColumn("database", (F.col("soc4").isin(list(database_soc)) & is_data_intensive).cast("int"))
        .withColumn("data_analytics", (F.col("soc4").isin(list(data_analytics_soc)) & is_data_intensive).cast("int"))
        .withColumn("any_data_intensive", is_data_intensive.cast("int"))
        .withColumn("year", F.year(F.to_date("date")))
        .withColumn("month", F.month(F.to_date("date")))
    )

    # Save job-level categories (Lazy check for existence)
    jc_base = ALL_PATHS[yr]["job_categories"]
    if not FORCE_RECOMPUTE:
        try:
            read_latest_version(jc_base)
            print(f"[SKIP] job_categories exists for {yr}")
            # We must load it if we skipped computation to support the next step? 
            # Actually, for the summary below, it's safer to just re-compute the aggregation 
            # from the features since aggregation is fast.
        except:
             safe_write_parquet_s3a(job_categories, jc_base, create_version=True, verify_read=False)
    else:
        safe_write_parquet_s3a(job_categories, jc_base, create_version=True, verify_read=False)

    # Aggregation: Occupation summary (SOC4)
    # Note: We rely on the dataframe 'job_categories' which is lazy. 
    # Spark will optimize the read from job_features directly.
    occ = (
        job_categories.groupBy("soc4")
        .agg(
            F.count("*").alias("total_jobs"),
            F.sum("data_entry").alias("data_entry_jobs"),
            F.sum("database").alias("database_jobs"),
            F.sum("data_analytics").alias("data_analytics_jobs"),
            F.sum("any_data_intensive").alias("any_data_intensive_jobs"),
        )
        .withColumn("data_entry_share", 100 * F.col("data_entry_jobs") / F.col("total_jobs"))
        .withColumn("database_share", 100 * F.col("database_jobs") / F.col("total_jobs"))
        .withColumn("data_analytics_share", 100 * F.col("data_analytics_jobs") / F.col("total_jobs"))
        .withColumn("total_data_share", 100 * F.col("any_data_intensive_jobs") / F.col("total_jobs"))
        .withColumn("year", F.lit(int(yr)))
    )

    os_base = ALL_PATHS[yr]["occupation_summary"]
    safe_write_parquet_s3a(occ, os_base, create_version=True, verify_read=False)

    print(f"[CLASSIFY] Year {yr} completed.")

Job classification + occ summary:   0%|          | 0/6 [00:00<?, ?it/s]


[CLASSIFY] Year 2020
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/job_categories/_v=20260218_193346


[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/occupation_summary/_v=20260218_193348


Job classification + occ summary:  17%|█▋        | 1/6 [00:04<00:23,  4.79s/it] 

[CLASSIFY] Year 2020 completed.

[CLASSIFY] Year 2021
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2021/job_categories/_v=20260218_193350


...

Job classification + occ summary:  83%|████████▎ | 5/6 [00:21<00:04,  4.18s/it] 

[CLASSIFY] Year 2024 completed.

[CLASSIFY] Year 2025
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/job_categories/_v=20260218_193407


[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/occupation_summary/_v=20260218_193409


Job classification + occ summary: 100%|██████████| 6/6 [00:24<00:00,  4.16s/it] 

[CLASSIFY] Year 2025 completed.


## Section 4: Economic Mapping & Sector Aggregation
### 4.1: Census-Based SIC Allocation
Maps occupations (SOC4) to industrial sectors (SIC) using Census employment weights:
- **Census Data Source**: Uses Census.csv with SOC4-level SIC distribution
- **Weight Calculation**: Builds normalized SOC4 → SIC mapping from Census employment data
- **Grouped SIC**: Aggregates detailed SIC2 codes into economic groupings (A, B-E, F, G-I, etc.)
- **Quality Checks**: Validates weight sums and identifies anomalies
- **Output**: Produces sector-level summaries with weighted data-intensity shares

In [57]:
# =============================================================================
# Census-based SOC->SIC allocation + sector summary (REPLACES synthetic block)
# =============================================================================
# What this block does:
# 1) Reads/Builds a SOC4 -> SIC_Code weight matrix from the Census table (CT21_0190 / your Census.csv)
#    - Census table is counts of workers by SOC4 across SIC2 industries.
# 2) Collapses SIC2 -> your grouped SIC_Code buckets that match your SUT table:
#      A, B-E, F, G-I, J, K, L, M-N, O-Q, R-T, U
# 3) Renormalises weights so that for each SOC4: sum_w = 1.0 (after grouping)
# 4) Applies those weights to your SOC4 occupation outputs (occupation_summary)
#    to produce sector totals and shares by SIC_Code, per year.
# 5) Writes a per-year sector summary output to:
#      ALL_PATHS[yr]["sector_summary_sic"]
#
# Notes:
# - This DOES NOT touch the expensive NLP step. It only re-allocates results after Step 6.
# - It avoids divide-by-zero by using safe divisions (when total_jobs == 0).
# - It replaces the old synthetic SIC field "SICSection_synth".
#
# Requirements:
# - You must have occupation_summary already written per year (Step 6).
# - You must have safe_write_parquet_s3a() and read_latest_version() defined earlier.
# - You must add *no new* ALL_PATHS keys if you re-use "sector_summary_sic".
# =============================================================================

from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from tqdm.auto import tqdm

# -------------------------
# CONFIG: Census mapping file
# -------------------------
# Use the CSV you created/exported (recommended). If you prefer the XLSX, convert to CSV first.
#CENSUS_CSV_PATH = CENSUS_CSV_PATH  # assumes you already defined it in your notebook parameters cell
# Example:
CENSUS_CSV_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/Census.csv"

# -------------------------
# Helper: SIC2 -> grouped SIC bucket matching your SUT table
# -------------------------
def sic2_to_grouped(col_sic2):
    # tolerant: trim, remove non-digits, then try_cast
    s = F.expr(f"try_cast(regexp_extract(trim({col_sic2}), '^(\\\\d{{2}})', 1) as int)")
    return (
        F.when((s >= 1) & (s <= 3), F.lit("A"))
         .when((s >= 5) & (s <= 9), F.lit("B-E"))     # B
         .when((s >= 10) & (s <= 33), F.lit("B-E"))   # C
         .when(s == 35, F.lit("B-E"))                 # D
         .when((s >= 36) & (s <= 39), F.lit("B-E"))   # E
         .when((s >= 41) & (s <= 43), F.lit("F"))
         .when((s >= 45) & (s <= 47), F.lit("G-I"))   # G
         .when((s >= 49) & (s <= 53), F.lit("G-I"))   # H
         .when((s >= 55) & (s <= 56), F.lit("G-I"))   # I
         .when((s >= 58) & (s <= 63), F.lit("J"))
         .when((s >= 64) & (s <= 66), F.lit("K"))
         .when(s == 68, F.lit("L"))
         .when((s >= 69) & (s <= 75), F.lit("M-N"))   # M
         .when((s >= 77) & (s <= 82), F.lit("M-N"))   # N
         .when(s == 84, F.lit("O-Q"))                 # O
         .when(s == 85, F.lit("O-Q"))                 # P
         .when((s >= 86) & (s <= 88), F.lit("O-Q"))   # Q
         .when((s >= 90) & (s <= 93), F.lit("R-T"))   # R
         .when((s >= 94) & (s <= 96), F.lit("R-T"))   # S
         .when((s >= 97) & (s <= 98), F.lit("R-T"))   # T
         .when(s == 99, F.lit("U"))
         .otherwise(F.lit(None))
    )

# -------------------------
# Build Census SOC4 -> grouped SIC weights
# -------------------------
# Expectation for Census.csv columns:
# - A first column like "4_digit_code_description" containing "1111 Chief executives ..."
# - Many SIC columns where each column name starts with "01 Crop and animal ..." etc
# If your first column name differs, adjust here:
CENSUS_SOC_DESC_COL = "4_digit_code_description"

c_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", False)
    .csv(CENSUS_CSV_PATH)
)

# Parse SOC4 + SOC title
c = (
    c_raw
    .withColumn("soc4", F.regexp_extract(F.col(CENSUS_SOC_DESC_COL), r"^(\d{4})", 1))
    .withColumn("soc_title", F.regexp_replace(F.col(CENSUS_SOC_DESC_COL), r"^\d{4}\s*", ""))
)

# Identify SIC columns (everything except the SOC descriptor column)
sic_cols = [x for x in c.columns if x != CENSUS_SOC_DESC_COL]

# Keep only what we need (soc4, title, SIC columns)
c = c.select("soc4", "soc_title", *sic_cols)

# Long-format: (soc4, soc_title, sic2_colname, n_raw)
stack_expr = "stack({n}, {pairs}) as (sic2_col, n_raw)".format(
    n=len(sic_cols),
    pairs=", ".join([f"'{col}', `{col}`" for col in sic_cols])
)
long = c.select("soc4", "soc_title", F.expr(stack_expr))

# Extract SIC2 code from column name + clean counts
long = (
    long
    .withColumn("sic2", F.regexp_extract(F.col("sic2_col"), r"^(\d{2})", 1))
    .withColumn("n_clean", F.regexp_replace(F.col("n_raw"), ",", ""))
    .withColumn("n", F.expr("coalesce(try_cast(n_clean as long), 0L)"))
    .drop("n_raw", "n_clean")
    .filter((F.col("soc4").isNotNull()) & (F.length("soc4") == 4) & (F.col("n") > 0))
)

# Collapse SIC2 -> grouped SIC_Code
long = long.withColumn("SIC_Code", sic2_to_grouped("sic2")).filter(F.col("SIC_Code").isNotNull())

# Aggregate counts at (soc4, grouped SIC_Code)
soc_sic_group = (
    long.groupBy("soc4", "SIC_Code")
        .agg(F.sum("n").alias("n"))
)

# Compute weights within SOC4 and RENORMALISE after grouping
soc_totals = soc_sic_group.groupBy("soc4").agg(F.sum("n").alias("soc4_total_n"))

soc_sic_group = (
    soc_sic_group
    .join(soc_totals, on="soc4", how="left")
    .withColumn(
        "w_soc4_SIC",
        F.when(F.col("soc4_total_n") > 0, F.col("n") / F.col("soc4_total_n")).otherwise(F.lit(0.0))
    )
)

# Optional sanity check: sums should be ~1
check = (
    soc_sic_group.groupBy("soc4")
    .agg(F.sum("w_soc4_SIC").alias("sum_w"))
)

print("\nCheck weight sums (SOC4 -> grouped SIC). Should be ~1 for each SOC4:")
check.orderBy(F.desc("sum_w")).show(20, truncate=False)

print("Lowest weight sums (to spot anomalies):")
check.orderBy(F.asc("sum_w")).show(20, truncate=False)

print("DONE: Census SOC->SIC mapping built (grouped + renormalised).")


# -------------------------
# Apply weights to occupation outputs (occupation_summary) -> sector summary
# -------------------------
def apply_census_sic_weights_to_occ(occ_df, weights_df):
    """
    occ_df: Spark DF with SOC4-level occupation outputs for a single year.
            Must include: soc4, total_jobs, data_entry_jobs, database_jobs, data_analytics_jobs, any_data_intensive_jobs
    weights_df: Spark DF with: soc4, SIC_Code, w_soc4_SIC (weights sum to 1 per soc4)

    Returns: sector summary DF grouped by SIC_Code with totals + shares (%)
    """
    occ_needed = occ_df.select(
        "soc4",
        F.col("total_jobs").cast("double").alias("total_jobs"),
        F.col("data_entry_jobs").cast("double").alias("data_entry_jobs"),
        F.col("database_jobs").cast("double").alias("database_jobs"),
        F.col("data_analytics_jobs").cast("double").alias("data_analytics_jobs"),
        F.col("any_data_intensive_jobs").cast("double").alias("any_data_intensive_jobs"),
    )

    # LEFT join so SOC4s missing in census weights stay visible (will go to NULL SIC -> drop later)
    alloc = (
        occ_needed.join(weights_df.select("soc4", "SIC_Code", "w_soc4_SIC"), on="soc4", how="left")
        .withColumn("w_soc4_SIC", F.coalesce(F.col("w_soc4_SIC"), F.lit(0.0)))
        .withColumn("SIC_Code", F.col("SIC_Code"))
        .withColumn("w_total_jobs", F.col("total_jobs") * F.col("w_soc4_SIC"))
        .withColumn("w_data_entry_jobs", F.col("data_entry_jobs") * F.col("w_soc4_SIC"))
        .withColumn("w_database_jobs", F.col("database_jobs") * F.col("w_soc4_SIC"))
        .withColumn("w_data_analytics_jobs", F.col("data_analytics_jobs") * F.col("w_soc4_SIC"))
        .withColumn("w_any_data_intensive_jobs", F.col("any_data_intensive_jobs") * F.col("w_soc4_SIC"))
    )

    sector = (
        alloc.filter(F.col("SIC_Code").isNotNull())
        .groupBy("SIC_Code")
        .agg(
            F.sum("w_total_jobs").alias("total_jobs"),
            F.sum("w_data_entry_jobs").alias("data_entry_jobs"),
            F.sum("w_database_jobs").alias("database_jobs"),
            F.sum("w_data_analytics_jobs").alias("data_analytics_jobs"),
            F.sum("w_any_data_intensive_jobs").alias("any_data_intensive_jobs"),
        )
    )

    # Safe shares (avoid divide-by-zero ANSI exception)
    sector = (
        sector
        .withColumn("data_entry_share", F.when(F.col("total_jobs") > 0, 100 * F.col("data_entry_jobs") / F.col("total_jobs")).otherwise(F.lit(0.0)))
        .withColumn("database_share", F.when(F.col("total_jobs") > 0, 100 * F.col("database_jobs") / F.col("total_jobs")).otherwise(F.lit(0.0)))
        .withColumn("data_analytics_share", F.when(F.col("total_jobs") > 0, 100 * F.col("data_analytics_jobs") / F.col("total_jobs")).otherwise(F.lit(0.0)))
        .withColumn("total_data_share", F.when(F.col("total_jobs") > 0, 100 * F.col("any_data_intensive_jobs") / F.col("total_jobs")).otherwise(F.lit(0.0)))
    )

    return sector


# -------------------------
# Per-year loop: read occ summary -> allocate -> write sector summary
# -------------------------
for yr in tqdm(years_to_process, desc="Census SIC allocation + sector summary"):
    print(f"\n[CENSUS-SIC] Year {yr}")

    # Read latest occupation summary for this year (SOC4-level)
    occ_df, occ_path = read_latest_version(ALL_PATHS[yr]["occupation_summary"])
    print(f"[CENSUS-SIC] Using occupation_summary from: {occ_path}")

    # Ensure correct year (defensive)
    if "year" in occ_df.columns:
        occ_df = occ_df.filter(F.col("year").cast("int") == F.lit(int(yr)))

    # Allocate SOC4 outputs into SIC_Code sectors using Census weights
    sector_census = apply_census_sic_weights_to_occ(occ_df, soc_sic_group).withColumn("year", F.lit(int(yr)))

    # Write to the SAME path key you already use downstream:
    # (this overwrites the synthetic sector summary conceptually, but still versioned safely)
    out_base = ALL_PATHS[yr]["sector_summary_sic"]

    if FORCE_RECOMPUTE:
        _written = safe_write_parquet_s3a(sector_census, out_base, create_version=True)
    else:
        try:
            _df, latest = read_latest_version(out_base)
            print(f"[SKIP] sector_summary_sic exists for {yr}: {latest}")
        except Exception:
            _written = safe_write_parquet_s3a(sector_census, out_base, create_version=True)

    print(f"[CENSUS-SIC] Year {yr} completed -> {out_base}")

print("\nDONE: Census-based sector summary written to sector_summary_sic (versioned).")


Check weight sums (SOC4 -> grouped SIC). Should be ~1 for each SOC4:


+----+------------------+
|soc4|sum_w             |
+----+------------------+
|5322|1.0000000000000004|
|2329|1.0000000000000002|
|9267|1.0000000000000002|
|2113|1.0000000000000002|
|3553|1.0000000000000002|
|3314|1.0000000000000002|
|6114|1.0000000000000002|
|6211|1.0000000000000002|
|8153|1.0000000000000002|
|5435|1.0000000000000002|
|9269|1.0000000000000002|
|2482|1.0000000000000002|
|2324|1.0000000000000002|
|5212|1.0000000000000002|
|4121|1.0000000000000002|
|9252|1.0000000000000002|
|2237|1.0000000000000002|
|6117|1.0000000000000002|
|9121|1.0000000000000002|
|5221|1.0000000000000002|
+----+------------------+
only showing top 20 rows
Lowest weight sums (to spot anomalies):
+----+------------------+
|soc4|sum_w             |
+----+------------------+
|2115|0.9999999999999998|
|2454|0.9999999999999998|
|3560|0.9999999999999998|
|2223|0.9999999999999998|
|2452|0.9999999999999998|
|4135|0.9999999999999998|
|5236|0.9999999999999998|
|6221|0.9999999999999999|
|8135|0.9999999999999999|

Census SIC allocation + sector summary:   0%|          | 0/6 [00:00<?, ?it/s]


[CENSUS-SIC] Year 2020
[CENSUS-SIC] Using occupation_summary from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/occupation_summary/_v=20260218_193348
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/sector_summary_sic/_v=20260218_193413


Census SIC allocation + sector summary:  17%|█▋        | 1/6 [00:02<00:14,  2.89s/it]

...

[CENSUS-SIC] Year 2023 completed -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2023/sector_summary_sic

[CENSUS-SIC] Year 2024
[CENSUS-SIC] Using occupation_summary from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/occupation_summary/_v=20260218_193405
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/sector_summary_sic/_v=20260218_193424


Census SIC allocation + sector summary:  83%|████████▎ | 5/6 [00:13<00:02,  2.70s/it]

[CENSUS-SIC] Year 2024 completed -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2024/sector_summary_sic

[CENSUS-SIC] Year 2025
[CENSUS-SIC] Using occupation_summary from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/occupation_summary/_v=20260218_193409
[WRITE] parquet -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/sector_summary_sic/_v=20260218_193426


Census SIC allocation + sector summary: 100%|██████████| 6/6 [00:16<00:00,  2.70s/it]

[CENSUS-SIC] Year 2025 completed -> /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2025/sector_summary_sic

DONE: Census-based sector summary written to sector_summary_sic (versioned).


## Section 5: VISUALIZATION & ANALYSIS
**Note**: This section can be run independently even with `FORCE_RECOMPUTE=False`. It reads pre-computed results and generates interactive charts.

### 5.1: Load Results for Visualization
Loads pre-computed `occupation_summary` and `sector_summary_sic_census` data from all years for analysis:
- **Data Consolidation**: Combines multi-year results into single DataFrames
- **Format Conversion**: Converts from Spark to Pandas for visualization
- **Type Casting**: Ensures proper numeric and categorical types
- **Ready for Analysis**: Prepared data for all downstream visualizations

In [58]:
# Load pre-computed occupation and sector summaries for all years
occ_frames = []
sec_frames = []

for yr in years_to_process:
    occ_df, occ_path = read_latest_version(ALL_PATHS[yr]["occupation_summary"])
    sec_df, sec_path = read_latest_version(ALL_PATHS[yr]["sector_summary_sic_census"])    
    occ_frames.append(occ_df.toPandas())
    sec_frames.append(sec_df.toPandas())

occupation_df = pd.concat(occ_frames, ignore_index=True)
sector_df = pd.concat(sec_frames, ignore_index=True)

occupation_df["year"] = occupation_df["year"].astype(int)
sector_df["year"] = sector_df["year"].astype(int)

print("✓ Loaded occupation_df:", occupation_df.shape)
print("✓ Loaded sector_df:", sector_df.shape)

✓ Loaded occupation_df: (66, 11)
✓ Loaded sector_df: (66, 11)


#### 5.1.1: Trends in Data-Intensive Jobs Over Time
Line chart showing the overall percentage of data-intensive job advertisements across the entire economy from 2020 to 2025.

In [59]:
yearly = (
    occupation_df.groupby("year")[["total_jobs","any_data_intensive_jobs"]]
    .sum()
    .reset_index()
)
yearly["data_intensive_share"] = 100 * yearly["any_data_intensive_jobs"] / yearly["total_jobs"]

fig = px.line(
    yearly.sort_values("year"),
    x="year",
    y="data_intensive_share",
    markers=True,
    title="Share of data-intensive job adverts over time",
    labels={"year":"Year", "data_intensive_share":"Data-intensive jobs (%)"}
)
fig.show()

#### 5.1.2: Top Data-Intensive Occupations by Year
Stacked bar chart showing the top 20 occupations with highest data intensity, broken down by data type (Entry, Database, Analytics) for each year.

In [60]:
top_n = 20
top_each_year = (
    occupation_df.sort_values(["year","total_data_share"], ascending=[True,False])
    .groupby("year")
    .head(top_n)
)

melted = top_each_year.melt(
    id_vars=["year","soc4"],
    value_vars=["data_entry_share","database_share","data_analytics_share"],
    var_name="category",
    value_name="share"
)

label_map = {
    "data_entry_share":"Data entry",
    "database_share":"Database",
    "data_analytics_share":"Data analytics"
}
melted["category_label"] = melted["category"].map(label_map)

fig = px.bar(
    melted,
    x="soc4",
    y="share",
    color="category_label",
    facet_col="year",
    facet_col_wrap=2,
    title=f"Top {top_n} occupations by data intensity — each year (stacked components)",
    labels={"soc4":"SOC (4-digit)", "share":"Share (%)", "category_label":"Layer"},
    height=900
)
fig.update_layout(barmode="stack")
fig.show()

#### 5.1.3: Sector Trends & Analysis

##### 5.1.3a: Sector Data-Intensity Over Time
Multi-line chart showing data-intensive job share trends for each industrial sector (classified by Census-based SIC grouping) from 2020 to 2025.

In [61]:
import plotly.express as px

fig = px.line(
    sector_df.sort_values(["year","SIC_Code"]),
    x="year",
    y="total_data_share",
    color="SIC_Code",
    markers=True,
    title="Census SIC group — data-intensive share over time",
    labels={"year":"Year","total_data_share":"Data-intensive share (%)","SIC_Code":"SIC group"}
)
fig.show()

##### 5.1.3b: Ad-hoc Analysis - Most Data-Related Jobs
Quick validation check to find specific job ads with the highest average semantic similarity to "data" using NLP vectors. Useful for qualitatively validating the model.

In [62]:
# NEW CELL: Find the most 'data-like' jobs regardless of threshold
from pyspark.sql import functions as F

# Load the job features we just wrote
df_features, _ = read_latest_version(ALL_PATHS[2024]["job_features"])

print("Top 10 jobs most related to 'data' by average similarity:")
df_features.filter(F.col("avg_sim_data").isNotNull()) \
           .orderBy(F.col("avg_sim_data").desc()) \
           .select("job_id", "avg_sim_data", "n_chunks_data", "top_chunks") \
           .show(10, truncate=False)

Top 10 jobs most related to 'data' by average similarity:
+------+------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------+
|job_id|avg_sim_data      |n_chunks_data|top_chunks                                                                                                                              |
+------+------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------+
|None  |0.4927734306880406|7            |[data analytics, sql database management, statistical modeling, dashboard visualization, machine learning, python programming, keywords]|
|None  |0.4927734306880406|7            |[data analytics, sql database management, statistical modeling, dashboard visualization, machine learning, python programming, keywords]|
|None  |0.4927734306880406|7            |[data 

### 4.2: Detailed Census SIC Mapping & Sector Summary (Comprehensive)
This cell provides a complete, standalone implementation of the Census-based SOC→SIC mapping:
- **Full Census Matrix Processing**: Reads Census.csv, builds SOC4×SIC2 weight matrix
- **SIC Grouping**: Maps detailed SIC2 codes to economic groupings
- **Weight Normalization**: Ensures weights sum to 1.0 per occupational category
- **Multi-Year Application**: Applies Census weights to each year's occupation summary independently
- **Validation**: Quality checks for missing sectors, weight sums, and anomalies
- **Independent Execution**: Can be run or skipped without affecting earlier steps

In [63]:
# -------------------------------------------------------------
# STEP 6 (CENSUS): Build SOC4 -> grouped SIC weights from Census,
# then allocate SOC4 occupation outputs to SIC sections (all years).
# -------------------------------------------------------------

from pyspark.sql import functions as F
from pyspark.sql.types import StringType, IntegerType, DoubleType

# ---- CONFIG (only change this in one place)
CENSUS_CSV_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/Census.csv"   # <-- update
WRITE_CENSUS_SECTOR_SUMMARY = True
FORCE_RECOMPUTE_CENSUS = False   # separate toggle for this step

# ---- Helper: map SIC2 (01..99) -> grouped SIC sections used by SUT table
def sic2_to_grouped(col_sic2_str):
    # robust: try cast (blank/non-numeric -> NULL)
    s = F.expr(f"try_cast({col_sic2_str} as int)")
    return (
        F.when((s >= 1) & (s <= 3), F.lit("A"))
         .when((s >= 5) & (s <= 9), F.lit("B-E"))      # B
         .when((s >= 10) & (s <= 33), F.lit("B-E"))    # C
         .when(s == 35, F.lit("B-E"))                  # D
         .when((s >= 36) & (s <= 39), F.lit("B-E"))    # E
         .when((s >= 41) & (s <= 43), F.lit("F"))
         .when((s >= 45) & (s <= 47), F.lit("G-I"))    # G
         .when((s >= 49) & (s <= 53), F.lit("G-I"))    # H
         .when((s >= 55) & (s <= 56), F.lit("G-I"))    # I
         .when((s >= 58) & (s <= 63), F.lit("J"))
         .when((s >= 64) & (s <= 66), F.lit("K"))
         .when(s == 68, F.lit("L"))
         .when((s >= 69) & (s <= 75), F.lit("M-N"))    # M
         .when((s >= 77) & (s <= 82), F.lit("M-N"))    # N
         .when(s == 84, F.lit("O-Q"))                  # O
         .when(s == 85, F.lit("O-Q"))                  # P
         .when((s >= 86) & (s <= 88), F.lit("O-Q"))    # Q
         .when((s >= 90) & (s <= 93), F.lit("R-T"))    # R
         .when((s >= 94) & (s <= 96), F.lit("R-T"))    # S
         .when((s >= 97) & (s <= 98), F.lit("R-T"))    # T
         .when(s == 99, F.lit("U"))
         .otherwise(F.lit(None))
    )

# -------------------------------------------------------------
# A) Read Census matrix (SOC rows x SIC2 columns) and build weights
# -------------------------------------------------------------
c = (
    spark.read
    .option("header", True)
    .option("inferSchema", False)
    .csv(CENSUS_CSV_PATH)
)

# Parse SOC4 + title from the row descriptor column
# Your file uses: "4_digit_code_description"
c = (
    c.withColumn("soc4", F.regexp_extract(F.col("4_digit_code_description"), r"^(\d{4})", 1))
     .withColumn("soc_title", F.regexp_replace(F.col("4_digit_code_description"), r"^\d{4}\s*", ""))
)

# Identify SIC columns (everything except the SOC descriptor)
sic_cols = [x for x in c.columns if x != "4_digit_code_description"]

# Long-format: SOC4, soc_title, sic2_colname, n_raw
stack_expr = "stack({n}, {pairs}) as (sic2_col, n_raw)".format(
    n=len(sic_cols),
    pairs=", ".join([f"'{col}', `{col}`" for col in sic_cols])
)

long = c.select("soc4", "soc_title", F.expr(stack_expr))

# Extract SIC2 from column name and clean counts
long = (
    long.withColumn("sic2", F.regexp_extract(F.col("sic2_col"), r"^(\d{2})", 1))
        .withColumn("sic2_desc", F.regexp_replace(F.col("sic2_col"), r"^\d{2}\s*", ""))
        .withColumn("n_clean", F.regexp_replace(F.col("n_raw"), ",", ""))
        .withColumn("n", F.expr("case when n_clean rlike '^[0-9]+$' then cast(n_clean as long) else 0 end"))
        .drop("n_raw", "n_clean")
)

# Drop blanks early so they never hit casts later
long = long.filter(F.col("soc4") != "").filter(F.col("sic2") != "")

# SOC4 totals (only across SIC2 columns that exist)
soc_totals = long.groupBy("soc4").agg(F.sum("n").alias("soc4_total"))

# Weight within SOC4 by SIC2
long = (
    long.join(soc_totals, on="soc4", how="left")
        .withColumn("w_soc4_sic2", F.when(F.col("soc4_total") > 0, F.col("n") / F.col("soc4_total")).otherwise(F.lit(0.0)))
)

# Map SIC2 -> grouped SIC sections and aggregate weights to grouped buckets
soc_sic_group = (
    long.withColumn("SIC_Code", sic2_to_grouped("sic2"))
        .filter(F.col("SIC_Code").isNotNull())
        .groupBy("soc4", "SIC_Code")
        .agg(F.sum("w_soc4_sic2").alias("w_soc4_SIC"))
)

# Renormalise within SOC4 so weights sum exactly to 1 (important!)
w_sum = soc_sic_group.groupBy("soc4").agg(F.sum("w_soc4_SIC").alias("sum_w"))
soc_sic_group = (
    soc_sic_group.join(w_sum, on="soc4", how="left")
        .withColumn("w_soc4_SIC", F.when(F.col("sum_w") > 0, F.col("w_soc4_SIC") / F.col("sum_w")).otherwise(F.lit(0.0)))
        .drop("sum_w")
)

# Quality check: weight sums should be ~1
check = soc_sic_group.groupBy("soc4").agg(F.sum("w_soc4_SIC").alias("sum_w"))

print("Check weight sums (SOC4 -> grouped SIC). Should be ~1 for each SOC4:")
check.orderBy(F.desc("sum_w")).show(20, truncate=False)

print("Lowest weight sums (to spot anomalies):")
check.orderBy(F.asc("sum_w")).show(20, truncate=False)

print("DONE: Census SOC->SIC mapping built (grouped + renormalised).")

# -------------------------------------------------------------
# B) Apply Census weights to each year’s occupation_summary (SOC4)
# -------------------------------------------------------------
def apply_census_sic_weights_to_occ(occ_df, weights_df):
    """
    occ_df: one row per SOC4 (from occupation_summary), with counts & shares
    weights_df: soc4, SIC_Code, w_soc4_SIC
    Returns: sector summary by SIC_Code with weighted counts and recomputed shares
    """

    # Join weights onto SOC4 occupation counts
    x = (
        occ_df.join(weights_df, on="soc4", how="left")
              .withColumn("w", F.coalesce(F.col("w_soc4_SIC"), F.lit(0.0)))
    )

    # Weighted counts (this is the critical bit: weights apply to COUNTS, not shares)
    x = (
        x.withColumn("w_total_jobs", F.col("total_jobs") * F.col("w"))
         .withColumn("w_data_entry_jobs", F.col("data_entry_jobs") * F.col("w"))
         .withColumn("w_database_jobs", F.col("database_jobs") * F.col("w"))
         .withColumn("w_data_analytics_jobs", F.col("data_analytics_jobs") * F.col("w"))
         .withColumn("w_any_data_intensive_jobs", F.col("any_data_intensive_jobs") * F.col("w"))
    )

    # Aggregate to SIC section
    sec = (
        x.groupBy("SIC_Code")
         .agg(
             F.sum("w_total_jobs").alias("total_jobs"),
             F.sum("w_data_entry_jobs").alias("data_entry_jobs"),
             F.sum("w_database_jobs").alias("database_jobs"),
             F.sum("w_data_analytics_jobs").alias("data_analytics_jobs"),
             F.sum("w_any_data_intensive_jobs").alias("any_data_intensive_jobs"),
         )
            .withColumn(
                "data_entry_share",
                F.when(F.col("total_jobs") > 0,
                    100 * F.col("data_entry_jobs") / F.col("total_jobs"))
                .otherwise(F.lit(0.0))
            )
            .withColumn(
                "database_share",
                F.when(F.col("total_jobs") > 0,
                    100 * F.col("database_jobs") / F.col("total_jobs"))
                .otherwise(F.lit(0.0))
            )
            .withColumn(
                "data_analytics_share",
                F.when(F.col("total_jobs") > 0,
                    100 * F.col("data_analytics_jobs") / F.col("total_jobs"))
                .otherwise(F.lit(0.0))
            )
            .withColumn(
                "total_data_share",
                F.when(F.col("total_jobs") > 0,
                    100 * F.col("any_data_intensive_jobs") / F.col("total_jobs"))
                .otherwise(F.lit(0.0))
            )
    )

    return sec

# Build sector summary for every year (2020-2025)
for yr in years_to_process:
    out_base = ALL_PATHS[yr]["sector_summary_sic_census"]

    if not FORCE_RECOMPUTE_CENSUS:
        try:
            _df, latest = read_latest_version(out_base)
            print(f"[SKIP] sector_summary_sic_census exists for {yr}: {latest}")
            continue
        except Exception:
            pass

    print(f"\n[CENSUS-SIC] Building sector summary for year {yr}...")

    occ_df, occ_path = read_latest_version(ALL_PATHS[yr]["occupation_summary"])
    occ_df = occ_df.filter(F.col("year") == F.lit(int(yr)))

    # Apply weights
    sector_census = apply_census_sic_weights_to_occ(occ_df, soc_sic_group).withColumn("year", F.lit(int(yr)))

    # Write
    safe_write_parquet_s3a(sector_census, out_base, create_version=True)

    print(f"[CENSUS-SIC] Year {yr} done -> {out_base}")

Check weight sums (SOC4 -> grouped SIC). Should be ~1 for each SOC4:
+----+------------------+
|soc4|sum_w             |
+----+------------------+
|6211|1.0000000000000004|
|6113|1.0000000000000002|
|1252|1.0000000000000002|
|2113|1.0000000000000002|
|3314|1.0000000000000002|
|2161|1.0000000000000002|
|4151|1.0000000000000002|
|1111|1.0000000000000002|
|5432|1.0000000000000002|
|9221|1.0000000000000002|
|9269|1.0000000000000002|
|5221|1.0000000000000002|
|2254|1.0000000000000002|
|2136|1.0000000000000002|
|5234|1.0000000000000002|
|2315|1.0000000000000002|
|4215|1.0000000000000002|
|3534|1.0000000000000002|
|4121|1.0000000000000002|
|8149|1.0000000000000002|
+----+------------------+
only showing top 20 rows
Lowest weight sums (to spot anomalies):
+----+------------------+
|soc4|sum_w             |
+----+------------------+
|1112|0.9999999999999997|
|9131|0.9999999999999997|
|8120|0.9999999999999997|
|2452|0.9999999999999997|
|1161|0.9999999999999998|
|8159|0.9999999999999998|
|7219|0.

## Section 6: Economic Valuation (OECD Methodology)
### 6.1: OECD Data Investment Valuation Model
Final step to estimate monetary value of data investment using the OECD 2023 methodology:

**Formula**: $Investment_{i} = \alpha_{i} \times Comp\_Emp_{i} \times Share_{Data, i}$

**Parameters**:
- $Share_{Data, i}$: Portion of data-intensive jobs in sector $i$ (from job ad classification)
- $Comp\_Emp_{i}$: Total compensation of employees in sector $i$ (from Supply-Use Tables, benchmark year)
- $\alpha_{i}$: OECD "markup" parameter with two scenarios:
  - **LOW**: Conservative constant α ≈ 1.5-1.6 for all sectors
  - **SECTOR**: Sector-specific α values (ranging 2.0-6.6) derived from National Accounts

**Output**: Data investment expressed as a percentage of GVA (Gross Value Added)

In [64]:
# ============================================================
# STEP 7 — OECD Valuation (Paper-faithful): Data investment from job-ad shares
# ============================================================
# What this does (exactly in line with OECD 2023 "role of data in jobs"):
# 1) Take sector-level data-intensity SHARES from job ads (your sector_summary_sic_census)
# 2) Join to one benchmark-year SUT (COMP_EMP + GVA) at matching sector aggregation
# 3) Convert shares -> proportions
# 4) Compute investment by data type using:  I_{d,i} = alpha_i * COMP_EMP_i * share_{d,i}
# 5) Produce investment as % of GVA
#
# IMPORTANT:
# - α here is the OECD "markup" parameter (NOT capitalisation rates 0–1).
# - We produce two scenarios (OECD-style bounds):
#     (a) LOW: constant alpha everywhere (e.g. 1.58)
#     (b) HIGH/SECTOR: sector-specific alphas you computed from national accounts
# - We do NOT use 0.25/0.50/0.75 in this paper’s method.
# ============================================================

from pyspark.sql import functions as F

# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------
SUT_YEAR = 2023
SUT_CSV_PATH = "/Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/SUT_TABLE.csv"

# OECD-style α scenarios
ALPHA_LOW = 1.58           # conservative constant
ALPHA_ECONOMY_AVG = 3.62   # your economy-wide weighted value (used as fallback / for outlier replacement)

alpha_map = {
    "A":   ALPHA_ECONOMY_AVG,  # replace outlier
    "B-E": 6.45,
    "F":   6.64,
    "G-I": 2.95,
    "J":   2.97,
    "K":   3.91,
    "L":   ALPHA_ECONOMY_AVG,  # replace outlier
    "M-N": 2.79,
    "O-Q": 2.07,
    "R-T": 3.06,
    "U":   ALPHA_ECONOMY_AVG,  # if present
}

ALPHA_DEFAULT = ALPHA_ECONOMY_AVG  # fallback if an unexpected SIC_Code appears

# Where your sector shares live (CENSUS-based)
SECTOR_SUMMARY_KEY = "sector_summary_sic_census"  # <- this is the one that should contain SIC_Code already

# ------------------------------------------------------------
# Helper: build alpha lookup column (Spark-safe)
# ------------------------------------------------------------
alpha_pairs = []
for k, v in alpha_map.items():
    alpha_pairs.extend([F.lit(k), F.lit(float(v))])
alpha_lookup = F.create_map(*alpha_pairs)

# ------------------------------------------------------------
# 1) READ SUT (benchmark year)
# ------------------------------------------------------------
sut_raw = (
    spark.read
    .option("header", True)
    .csv(SUT_CSV_PATH)
)

print("\n[STEP 7] SUT raw columns:")
print(sut_raw.columns)

sut = (
    sut_raw
    .withColumn("year", F.col("year").cast("int"))
    .withColumn("SIC_Code", F.upper(F.trim(F.col("SIC_Code").cast("string"))))
    .withColumn("GVA_basic_prices", F.col("GVA_basic_prices").cast("double"))
    .withColumn("COMP_EMP", F.col("COMP_EMP").cast("double"))
    .filter(F.col("year") == F.lit(int(SUT_YEAR)))
)

print(f"\n[STEP 7] SUT filtered to year={SUT_YEAR}. Sample:")
sut.select("year", "SIC_Code", "GVA_basic_prices", "COMP_EMP").show(20, truncate=False)

# Quick sanity: sectors present in SUT
sut_sectors = [r["SIC_Code"] for r in sut.select("SIC_Code").distinct().collect()]
print("\n[STEP 7] SUT sectors present:", sorted(sut_sectors))

# ------------------------------------------------------------
# 2) LOAD SECTOR SUMMARY (ALL YEARS) — Census-based SIC allocation
# ------------------------------------------------------------
sec_frames = []
for yr in years_to_process:
    df, path_used = read_latest_version(ALL_PATHS[yr][SECTOR_SUMMARY_KEY])
    print(f"[STEP 7] Loaded {SECTOR_SUMMARY_KEY} for {yr} from: {path_used}")
    sec_frames.append(df)

sector_all = sec_frames[0]
for df in sec_frames[1:]:
    sector_all = sector_all.unionByName(df, allowMissingColumns=True)

print("\n[STEP 7] Sector summary columns (unioned):")
print(sector_all.columns)

# Defensive: ensure standard column names exist
required_cols = {"year", "SIC_Code", "total_jobs", "data_entry_share", "database_share", "data_analytics_share", "total_data_share"}
missing = sorted(list(required_cols - set(sector_all.columns)))
if missing:
    raise ValueError(
        f"[STEP 7] Missing required columns in {SECTOR_SUMMARY_KEY}: {missing}\n"
        f"Available columns: {sector_all.columns}\n"
        f"Tip: confirm you are reading sector_summary_sic_census (not sector_summary_sic)."
    )

sector_all = (
    sector_all
    .withColumn("year", F.col("year").cast("int"))
    .withColumn("SIC_Code", F.upper(F.trim(F.col("SIC_Code").cast("string"))))
    .withColumn("total_jobs", F.col("total_jobs").cast("double"))
    .withColumn("data_entry_share", F.col("data_entry_share").cast("double"))
    .withColumn("database_share", F.col("database_share").cast("double"))
    .withColumn("data_analytics_share", F.col("data_analytics_share").cast("double"))
    .withColumn("total_data_share", F.col("total_data_share").cast("double"))
)

print("\n[STEP 7] Sector summary sample (after casting):")
sector_all.select("year","SIC_Code","total_jobs","data_entry_share","database_share","data_analytics_share","total_data_share").show(20, truncate=False)

# Sanity: sectors present in sector summary
sec_sectors = [r["SIC_Code"] for r in sector_all.select("SIC_Code").distinct().collect()]
print("\n[STEP 7] Sector-summary sectors present:", sorted(sec_sectors))

# ------------------------------------------------------------
# 3) WEIGHTED RE-AGGREGATION (only needed if duplicates exist per year+SIC_Code)
# ------------------------------------------------------------
# If your sector_summary already has exactly one row per (year, SIC_Code), this is harmless.
# It recomputes shares using total_jobs as weights to be safe.
sector_grp = (
    sector_all
    .groupBy("year", "SIC_Code")
    .agg(
        F.sum("total_jobs").alias("total_jobs"),

        (F.sum(F.col("data_entry_share") * F.col("total_jobs")) /
         F.when(F.sum("total_jobs") > 0, F.sum("total_jobs")).otherwise(F.lit(1.0))
        ).alias("data_entry_share"),

        (F.sum(F.col("database_share") * F.col("total_jobs")) /
         F.when(F.sum("total_jobs") > 0, F.sum("total_jobs")).otherwise(F.lit(1.0))
        ).alias("database_share"),

        (F.sum(F.col("data_analytics_share") * F.col("total_jobs")) /
         F.when(F.sum("total_jobs") > 0, F.sum("total_jobs")).otherwise(F.lit(1.0))
        ).alias("data_analytics_share"),

        (F.sum(F.col("total_data_share") * F.col("total_jobs")) /
         F.when(F.sum("total_jobs") > 0, F.sum("total_jobs")).otherwise(F.lit(1.0))
        ).alias("total_data_share"),
    )
)

print("\n[STEP 7] Sector grouped sample:")
sector_grp.orderBy("year", "SIC_Code").show(20, truncate=False)

# ------------------------------------------------------------
# 4) JOIN TO SUT (benchmark values applied to all years)
# ------------------------------------------------------------
valued = (
    sector_grp
    .join(
        sut.select("SIC_Code", "GVA_basic_prices", "COMP_EMP"),
        on="SIC_Code",
        how="inner"
    )
)

print("\n[STEP 7] After join to SUT — sample:")
valued.select("year","SIC_Code","COMP_EMP","GVA_basic_prices","total_jobs","total_data_share").orderBy("year","SIC_Code").show(20, truncate=False)

# Diagnostics: which sectors got dropped due to join?
joined_sectors = {r["SIC_Code"] for r in valued.select("SIC_Code").distinct().collect()}
dropped = sorted(set(sec_sectors) - set(joined_sectors))
if dropped:
    print("\n[STEP 7][WARNING] These sector(s) exist in sector_summary but NOT in SUT, so they were dropped in valuation:")
    print(dropped)
else:
    print("\n[STEP 7] All sector_summary SIC_Codes matched SUT.")

# ------------------------------------------------------------
# 5) Convert shares (%) -> proportions
# ------------------------------------------------------------
valued = (
    valued
    .withColumn("p_data_entry", F.col("data_entry_share") / F.lit(100.0))
    .withColumn("p_database", F.col("database_share") / F.lit(100.0))
    .withColumn("p_data_analytics", F.col("data_analytics_share") / F.lit(100.0))
)

# ------------------------------------------------------------
# 6) Attach α (LOW constant + SECTOR lookup)
# ------------------------------------------------------------
valued = (
    valued
    .withColumn("alpha_low", F.lit(float(ALPHA_LOW)))
    .withColumn("alpha_sector", F.coalesce(alpha_lookup[F.col("SIC_Code")], F.lit(float(ALPHA_DEFAULT))))
)

print("\n[STEP 7] α values attached — sample:")
valued.select("SIC_Code","alpha_low","alpha_sector").distinct().orderBy("SIC_Code").show(50, truncate=False)

# ------------------------------------------------------------
# 7) OECD valuation equation (paper-faithful):
#     Investment_{d,i} = alpha_i * COMP_EMP_i * share_{d,i}
# ------------------------------------------------------------

# LOW scenario
valued = (
    valued
    .withColumn("inv_entry_low",    F.col("alpha_low")    * F.col("COMP_EMP") * F.col("p_data_entry"))
    .withColumn("inv_database_low", F.col("alpha_low")    * F.col("COMP_EMP") * F.col("p_database"))
    .withColumn("inv_analytics_low",F.col("alpha_low")    * F.col("COMP_EMP") * F.col("p_data_analytics"))
    .withColumn("total_investment_low", F.col("inv_entry_low") + F.col("inv_database_low") + F.col("inv_analytics_low"))
)

# SECTOR scenario
valued = (
    valued
    .withColumn("inv_entry_sector",    F.col("alpha_sector") * F.col("COMP_EMP") * F.col("p_data_entry"))
    .withColumn("inv_database_sector", F.col("alpha_sector") * F.col("COMP_EMP") * F.col("p_database"))
    .withColumn("inv_analytics_sector",F.col("alpha_sector") * F.col("COMP_EMP") * F.col("p_data_analytics"))
    .withColumn("total_investment_sector", F.col("inv_entry_sector") + F.col("inv_database_sector") + F.col("inv_analytics_sector"))
)

# ------------------------------------------------------------
# 8) Express as % of GVA
# ------------------------------------------------------------
valued = (
    valued
    .withColumn(
        "investment_share_gva_low",
        F.when(F.col("GVA_basic_prices") > 0, 100.0 * F.col("total_investment_low") / F.col("GVA_basic_prices")).otherwise(F.lit(0.0))
    )
    .withColumn(
        "investment_share_gva_sector",
        F.when(F.col("GVA_basic_prices") > 0, 100.0 * F.col("total_investment_sector") / F.col("GVA_basic_prices")).otherwise(F.lit(0.0))
    )
)

# ------------------------------------------------------------
# 9) Final output table
# ------------------------------------------------------------
final_cols = [
    "year", "SIC_Code",
    "GVA_basic_prices", "COMP_EMP",
    "total_jobs",
    "data_entry_share", "database_share", "data_analytics_share", "total_data_share",
    "alpha_low", "alpha_sector",
    "total_investment_low", "investment_share_gva_low",
    "total_investment_sector", "investment_share_gva_sector",
]

final_valuation = valued.select(*final_cols).orderBy("year","SIC_Code")

print("\n[STEP 7] STEP 7 COMPLETE — OECD-style valuation output (LOW + SECTOR α). Top rows:")
final_valuation.show(50, truncate=False)

# Optional: quick economy-wide totals per year (helps sanity-check small synthetic samples)
econ = (
    final_valuation
    .groupBy("year")
    .agg(
        F.sum("total_investment_low").alias("econ_total_investment_low"),
        F.sum("total_investment_sector").alias("econ_total_investment_sector"),
        F.sum("GVA_basic_prices").alias("econ_total_gva"),
    )
    .withColumn("econ_share_gva_low", F.when(F.col("econ_total_gva") > 0, 100.0 * F.col("econ_total_investment_low") / F.col("econ_total_gva")).otherwise(F.lit(0.0)))
    .withColumn("econ_share_gva_sector", F.when(F.col("econ_total_gva") > 0, 100.0 * F.col("econ_total_investment_sector") / F.col("econ_total_gva")).otherwise(F.lit(0.0)))
    .orderBy("year")
)

print("\n[STEP 7] Economy-wide totals per year (sanity check):")
econ.show(50, truncate=False)


[STEP 7] SUT raw columns:
['year', 'SICSection_Full_name', 'SIC_Code', 'GVA_basic_prices', 'COMP_EMP']

[STEP 7] SUT filtered to year=2023. Sample:
+----+--------+----------------+--------+
|year|SIC_Code|GVA_basic_prices|COMP_EMP|
+----+--------+----------------+--------+
|2023|A       |17839.0         |4840.0  |
|2023|B-E     |337889.0        |157871.0|
|2023|F       |148457.0        |60509.0 |
|2023|G-I     |401728.0        |265598.0|
|2023|J       |152297.0        |100567.0|
|2023|K       |204046.0        |99037.0 |
|2023|L       |346849.0        |20114.0 |
|2023|M-N     |340424.0        |215043.0|
|2023|O-Q     |473216.0        |366475.0|
|2023|R-T     |74645.0         |39724.0 |
+----+--------+----------------+--------+


[STEP 7] SUT sectors present: ['A', 'B-E', 'F', 'G-I', 'J', 'K', 'L', 'M-N', 'O-Q', 'R-T']
[STEP 7] Loaded sector_summary_sic_census for 2020 from: /Users/saurabhkumar/Desktop/OECD_PYSPARK_LOCAL/data/parquet_OECD/processed_data/2020/sector_summary_sic_census/_v

### 6.2: Valuation Results & Visualizations
Generates 4 key charts showing data investment estimates:
1. **Economy-wide**: Total data investment as % of GVA over time (LOW vs SECTOR scenarios)
2. **Sector-level (LOW)**: Industry-specific investment share using constant α
3. **Sector-level (SECTOR)**: Industry-specific investment share using sector-specific α values
4. **Investment Levels**: Absolute magnitude of data investment by year and scenario

In [65]:
# ============================================================
# STEP 8 — Visualisations (OECD-style)
# ============================================================
# Requires: final_valuation (from Step 7)
# Produces:
# 1) Economy-wide time series (% of GVA): LOW vs SECTOR
# 2) Sector-level time series (% of GVA): LOW and SECTOR
# 3) Economy-wide totals (levels): total investment LOW vs SECTOR
# ============================================================

import pandas as pd
import plotly.express as px

print("\n[STEP 8] final_valuation columns:")
print(final_valuation.columns)

print("\n[STEP 8] final_valuation preview:")
final_valuation.show(20, truncate=False)

# ------------------------------------------------------------
# 1) Economy-wide series (already computed as econ in Step 7)
#    If econ exists, use it; otherwise rebuild it.
# ------------------------------------------------------------
try:
    econ
    print("\n[STEP 8] Using existing econ dataframe from Step 7.")
except NameError:
    print("\n[STEP 8] econ not found — rebuilding from final_valuation.")
    econ = (
        final_valuation
        .groupBy("year")
        .agg(
            F.sum("total_investment_low").alias("econ_total_investment_low"),
            F.sum("total_investment_sector").alias("econ_total_investment_sector"),
            F.sum("GVA_basic_prices").alias("econ_total_gva"),
        )
        .withColumn(
            "econ_share_gva_low",
            F.when(F.col("econ_total_gva") > 0, 100.0 * F.col("econ_total_investment_low") / F.col("econ_total_gva")).otherwise(F.lit(0.0))
        )
        .withColumn(
            "econ_share_gva_sector",
            F.when(F.col("econ_total_gva") > 0, 100.0 * F.col("econ_total_investment_sector") / F.col("econ_total_gva")).otherwise(F.lit(0.0))
        )
        .orderBy("year")
    )

print("\n[STEP 8] Economy-wide series preview:")
econ.show(50, truncate=False)

econ_pd = econ.toPandas()
econ_pd["year"] = econ_pd["year"].astype(int)

# Long format for plotting (LOW vs SECTOR)
econ_long = econ_pd.melt(
    id_vars=["year"],
    value_vars=["econ_share_gva_low", "econ_share_gva_sector"],
    var_name="scenario",
    value_name="data_investment_share_gva"
)

scenario_name = {
    "econ_share_gva_low": "LOW α (constant)",
    "econ_share_gva_sector": "SECTOR α (your estimates)"
}
econ_long["scenario"] = econ_long["scenario"].map(scenario_name)

fig1 = px.line(
    econ_long.sort_values(["year", "scenario"]),
    x="year",
    y="data_investment_share_gva",
    color="scenario",
    markers=True,
    title="Economy-wide data investment as % of GVA (benchmark SUT applied)",
    labels={"data_investment_share_gva": "Data investment (% of GVA)", "year": "Year", "scenario": "Scenario"},
)
fig1.show()

# ------------------------------------------------------------
# 2) Sector-level time series (% of GVA)
# ------------------------------------------------------------
sec_pd = final_valuation.select(
    "year", "SIC_Code",
    "investment_share_gva_low",
    "investment_share_gva_sector",
    "total_investment_low",
    "total_investment_sector"
).toPandas()

sec_pd["year"] = sec_pd["year"].astype(int)

# LOW by sector
fig2 = px.line(
    sec_pd.sort_values(["year", "SIC_Code"]),
    x="year",
    y="investment_share_gva_low",
    color="SIC_Code",
    markers=True,
    title="Sector data investment as % of GVA (LOW α constant)",
    labels={"investment_share_gva_low": "Data investment (% of GVA)", "year": "Year", "SIC_Code": "SIC sector"},
)
fig2.show()

# SECTOR alpha by sector
fig3 = px.line(
    sec_pd.sort_values(["year", "SIC_Code"]),
    x="year",
    y="investment_share_gva_sector",
    color="SIC_Code",
    markers=True,
    title="Sector data investment as % of GVA (SECTOR α)",
    labels={"investment_share_gva_sector": "Data investment (% of GVA)", "year": "Year", "SIC_Code": "SIC sector"},
)
fig3.show()

# ------------------------------------------------------------
# 3) Economy-wide totals (levels)
# ------------------------------------------------------------
econ_levels = econ_pd[["year", "econ_total_investment_low", "econ_total_investment_sector"]].copy()
econ_levels_long = econ_levels.melt(
    id_vars=["year"],
    value_vars=["econ_total_investment_low", "econ_total_investment_sector"],
    var_name="scenario",
    value_name="total_investment"
)
scenario_name2 = {
    "econ_total_investment_low": "LOW α (constant)",
    "econ_total_investment_sector": "SECTOR α (your estimates)"
}
econ_levels_long["scenario"] = econ_levels_long["scenario"].map(scenario_name2)

fig4 = px.line(
    econ_levels_long.sort_values(["year","scenario"]),
    x="year",
    y="total_investment",
    color="scenario",
    markers=True,
    title="Economy-wide data investment (levels) — LOW vs SECTOR α",
    labels={"total_investment": "Total data investment (units of SUT values)", "year":"Year", "scenario":"Scenario"},
)
fig4.show()

print("\n[STEP 8] DONE — charts rendered.")


[STEP 8] final_valuation columns:
['year', 'SIC_Code', 'GVA_basic_prices', 'COMP_EMP', 'total_jobs', 'data_entry_share', 'database_share', 'data_analytics_share', 'total_data_share', 'alpha_low', 'alpha_sector', 'total_investment_low', 'investment_share_gva_low', 'total_investment_sector', 'investment_share_gva_sector']

[STEP 8] final_valuation preview:
+----+--------+----------------+--------+------------------+----------------+------------------+--------------------+------------------+---------+------------+--------------------+------------------------+-----------------------+---------------------------+
|year|SIC_Code|GVA_basic_prices|COMP_EMP|total_jobs        |data_entry_share|database_share    |data_analytics_share|total_data_share  |alpha_low|alpha_sector|total_investment_low|investment_share_gva_low|total_investment_sector|investment_share_gva_sector|
+----+--------+----------------+--------+------------------+----------------+------------------+--------------------+---------


[STEP 8] DONE — charts rendered.


### 6.3: Alpha Sensitivity Analysis (Low vs Central vs Sector)
Performs a comprehensive sensitivity analysis with three α scenarios:
- **LOW** (α = 1.5): Conservative constant markup across all sectors
- **CENTRAL** (α = 3.62): Middle-ground constant markup
- **SECTOR**: Sector-specific α values based on National Accounts calibration

**Output**: Generates sector-level and economy-wide comparison charts showing how different α assumptions affect data investment estimates. Provides bounds for final estimates.

In [66]:
# ============================================================
# STEP 9 — Alpha Sensitivity (Low vs Sector vs Central)
# Requires: final_valuation from Step 8
# ============================================================

from pyspark.sql import functions as F
import pandas as pd
import plotly.express as px

print("\n[STEP 9] final_valuation columns:")
print(final_valuation.columns)

# ------------------------------------------------------------
# 1) Define α scenarios (exactly what you want)
# ------------------------------------------------------------
ALPHA_LOW     = 1.50
ALPHA_CENTRAL = 3.62

# Your sector α map (grouped SIC)
alpha_map = {
    "A": 3.62,
    "B-E": 6.45,
    "F": 6.64,
    "G-I": 2.95,
    "J": 2.97,
    "K": 3.91,
    "L": 3.62,
    "M-N": 2.79,
    "O-Q": 2.07,
    "R-T": 3.06
}
ALPHA_DEFAULT = 3.62

print("\n[STEP 9] Alpha scenarios used:")
print(f"  - Low (constant): {ALPHA_LOW}")
print("  - Sector-specific: alpha_map by SIC_Code (with default = 3.62)")
print(f"  - Central (constant): {ALPHA_CENTRAL}")

# Build Spark map expression for sector α lookup
alpha_expr = F.create_map(*[F.lit(x) for kv in alpha_map.items() for x in kv])

# ------------------------------------------------------------
# 2) Base frame: compute p_total_data from total_data_share (%)
# ------------------------------------------------------------
base = (
    final_valuation
    .select(
        "year", "SIC_Code",
        "GVA_basic_prices", "COMP_EMP",
        "total_data_share"
    )
    .withColumn("p_total_data", F.col("total_data_share") / F.lit(100.0))
)

print("\n[STEP 9] Base preview (year, SIC_Code, p_total_data):")
base.select("year","SIC_Code","total_data_share","p_total_data").orderBy("year","SIC_Code").show(10, truncate=False)

# ------------------------------------------------------------
# 3) Compute three scenario investments at SECTOR×YEAR level
#    inv = α * COMP_EMP * p_total_data
# ------------------------------------------------------------
sector_alpha = F.coalesce(alpha_expr[F.col("SIC_Code")], F.lit(ALPHA_DEFAULT))

sector_sens = (
    base
    .withColumn("inv_low",     F.lit(ALPHA_LOW)     * F.col("COMP_EMP") * F.col("p_total_data"))
    .withColumn("inv_sector",  sector_alpha         * F.col("COMP_EMP") * F.col("p_total_data"))
    .withColumn("inv_central", F.lit(ALPHA_CENTRAL) * F.col("COMP_EMP") * F.col("p_total_data"))
    .withColumn("share_low",
                F.when(F.col("GVA_basic_prices") > 0, 100.0 * F.col("inv_low") / F.col("GVA_basic_prices"))
                 .otherwise(F.lit(None)))
    .withColumn("share_sector",
                F.when(F.col("GVA_basic_prices") > 0, 100.0 * F.col("inv_sector") / F.col("GVA_basic_prices"))
                 .otherwise(F.lit(None)))
    .withColumn("share_central",
                F.when(F.col("GVA_basic_prices") > 0, 100.0 * F.col("inv_central") / F.col("GVA_basic_prices"))
                 .otherwise(F.lit(None)))
)

print("\n[STEP 9] Sector×Year sensitivity preview:")
sector_sens.select(
    "year","SIC_Code","total_data_share","COMP_EMP","GVA_basic_prices",
    "inv_low","share_low","inv_sector","share_sector","inv_central","share_central"
).orderBy("year","SIC_Code").show(20, truncate=False)

# ------------------------------------------------------------
# 4) Economy-wide per year (sum investment / sum GVA)
# ------------------------------------------------------------
econ = (
    sector_sens
    .groupBy("year")
    .agg(
        F.sum("inv_low").alias("inv_low_total"),
        F.sum("inv_sector").alias("inv_sector_total"),
        F.sum("inv_central").alias("inv_central_total"),
        F.sum("GVA_basic_prices").alias("GVA_total")
    )
    .withColumn("share_low_total",
                F.when(F.col("GVA_total") > 0, 100.0 * F.col("inv_low_total") / F.col("GVA_total"))
                 .otherwise(F.lit(None)))
    .withColumn("share_sector_total",
                F.when(F.col("GVA_total") > 0, 100.0 * F.col("inv_sector_total") / F.col("GVA_total"))
                 .otherwise(F.lit(None)))
    .withColumn("share_central_total",
                F.when(F.col("GVA_total") > 0, 100.0 * F.col("inv_central_total") / F.col("GVA_total"))
                 .otherwise(F.lit(None)))
    .orderBy("year")
)

print("\n[STEP 9] Economy-wide table (Spark):")
econ.show(50, truncate=False)

# ------------------------------------------------------------
# 5) Plot economy-wide time series (3 lines)
# ------------------------------------------------------------
econ_pd = econ.toPandas()
econ_pd["year"] = econ_pd["year"].astype(int)

econ_long = econ_pd.melt(
    id_vars=["year"],
    value_vars=["share_low_total","share_sector_total","share_central_total"],
    var_name="scenario",
    value_name="data_investment_share_of_GVA"
)

scenario_label = {
    "share_low_total": "Low (α=1.5)",
    "share_sector_total": "Sector-specific α",
    "share_central_total": "Central (α=3.62)"
}
econ_long["scenario"] = econ_long["scenario"].map(scenario_label)

fig1 = px.line(
    econ_long,
    x="year",
    y="data_investment_share_of_GVA",
    color="scenario",
    markers=True,
    title="Data investment as % of GVA — α sensitivity (Low vs Sector vs Central)",
    labels={"data_investment_share_of_GVA":"% of GVA", "scenario":"Scenario"}
)
fig1.show()

# ------------------------------------------------------------
# 6) Optional: sector heatmap for the sector-specific scenario
# ------------------------------------------------------------
sec_pd = (
    sector_sens
    .select("year","SIC_Code","share_sector")
    .toPandas()
)
sec_pd["year"] = sec_pd["year"].astype(int)

fig2 = px.density_heatmap(
    sec_pd,
    x="year",
    y="SIC_Code",
    z="share_sector",
    title="Sector data investment share of GVA — Sector-specific α scenario",
    labels={"share_sector":"% of GVA"}
)
fig2.show()


[STEP 9] final_valuation columns:
['year', 'SIC_Code', 'GVA_basic_prices', 'COMP_EMP', 'total_jobs', 'data_entry_share', 'database_share', 'data_analytics_share', 'total_data_share', 'alpha_low', 'alpha_sector', 'total_investment_low', 'investment_share_gva_low', 'total_investment_sector', 'investment_share_gva_sector']

[STEP 9] Alpha scenarios used:
  - Low (constant): 1.5
  - Sector-specific: alpha_map by SIC_Code (with default = 3.62)
  - Central (constant): 3.62

[STEP 9] Base preview (year, SIC_Code, p_total_data):
+----+--------+------------------+-------------------+
|year|SIC_Code|total_data_share  |p_total_data       |
+----+--------+------------------+-------------------+
|2020|A       |29.09373709160188 |0.2909373709160188 |
|2020|B-E     |24.31317663966131 |0.2431317663966131 |
|2020|F       |12.238490156100967|0.12238490156100967|
|2020|G-I     |39.90144534928891 |0.3990144534928891 |
|2020|J       |60.62255826916801 |0.6062255826916801 |
|2020|K       |75.01405759301262